## This notebook inputs the missing values using KNN

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [2]:
INPUT, OUTPUT = 'output/\CompleteWeeklyIndexes', 'output/\KNNWeeklyImputed'

In [3]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [7]:
df_test.head()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2015-01-02,2058.199951,2.708700e+09,17.790001,0.0,17832.990234,76270000.0,4230.240234,1.435150e+09,NaN,NaN,...,23721.300781,1.801713e+09,835.969971,333050000.0,NaN,0.60,NaN,NaN,NaN,NaN
2015-01-09,2044.810059,3.364140e+09,17.549999,0.0,17737.369141,93390000.0,4213.279785,1.715830e+09,17197.730469,155200.0,...,23919.949219,2.334810e+09,824.390015,499788100.0,NaN,0.91,NaN,NaN,NaN,NaN
2015-01-16,2019.420044,4.056410e+09,20.950001,0.0,17511.570313,140480000.0,4142.140137,1.970520e+09,16864.160156,171100.0,...,24103.519531,1.716810e+09,862.820007,592856800.0,NaN,0.00,NaN,NaN,NaN,NaN
2015-01-23,2051.820068,3.573560e+09,16.660000,0.0,17672.599609,97110000.0,4278.140137,1.653830e+09,17511.750000,132500.0,...,24850.449219,2.375624e+09,911.729980,804870700.0,NaN,0.00,NaN,NaN,NaN,NaN
2015-01-30,1994.989990,4.568650e+09,20.969999,0.0,17164.949219,168560000.0,4148.430176,2.264230e+09,17674.390625,173500.0,...,24507.050781,1.438942e+09,906.280029,545062800.0,NaN,0.00,NaN,NaN,NaN,NaN


In [5]:
#Scaled data has zero mean and unit variance
scaler = preprocessing.StandardScaler()
scaler.fit(df)
array = scaler.transform(df)
array_test = scaler.transform(df_test)
df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)

imp = KNNImputer(n_neighbors=5, weights="uniform")
imp.fit(df_scaled)
df_imputed = imp.transform(df_scaled)
df_test_imputed = imp.transform(df_test_scaled)
df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
# Descaling the data
array_2 = scaler.inverse_transform(df_final)
df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

array_test_2 = scaler.inverse_transform(df_test_final)
df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
df_final2.to_csv(OUTPUT+'_0_training.csv', index = True)
df_test_final2.to_csv(OUTPUT+'_0_test.csv', index = True)